In [ ]:
""" Initialize, import libraries and load data, clean data """

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm

from itertools import combinations
from scipy import stats
from statsmodels.tsa.api import VAR
from sklearn.metrics import r2_score

import ast
import pickle

import utils
import var_decomposition


output_path = 'rmarkdown/run4-mimicking_addl/'


# load market data
files_to_load = [
    'BAA10Y', # default spread between BAA-rated corporate bonds and 10-year Treasury bonds
    'T10Y3M', # term spread between 10-year Treasury bonds and 3-month Treasury bills
    'DTB4WK', # 4-week Treasury bill rate - used as a proxy for the risk-free rate
    'DTB3' # 3-month Treasury bill rate
]
collected_market_data = []
for filename in files_to_load:
    data = pd.read_csv(f'data/{filename}.csv')
    data['observation_date'] = pd.to_datetime(data['observation_date']) + pd.offsets.MonthEnd(0) #- pd.offsets.Day(1)
    data = data.rename(columns={
        'observation_date': 'Date',
    }).set_index('Date')
    collected_market_data.append(data)

df_market_data = pd.concat(collected_market_data, axis=1)
df_market_data['BAA10Y'] /= 100
df_market_data['T10Y3M'] /= 100
df_market_data['DTB3'] /= 100
df_market_data['DTB4WK_LOG'] = np.log(1 + df_market_data['DTB4WK'] / 100) # make percentage, take log, add 1 to avoid log(0)
df_market_data['DTB4WK'] /= 100

CAPE = pd.read_excel('data/ie_data.xls', sheet_name='Data', skiprows=7, header=0)[['Date', 'CAPE']]
CAPE = CAPE.dropna()

date_year_month = CAPE['Date'].astype(str).str.split('.')
CAPE['Date'] = pd.to_datetime(date_year_month.apply(lambda x: f"{x[0]}.{x[1] if len(x[1]) == 2 else f'{x[1]}0'}"), format='%Y.%m') # fix october month which is parsed as YYYY.01 instead of YYYY.10
CAPE['Date'] = CAPE['Date'] + pd.offsets.MonthEnd(0)
CAPE['CAPE'] = np.log(CAPE['CAPE'])  # Taking the log of CAPE for normalization
CAPE = CAPE.set_index('Date')

df_market_data = df_market_data.join(CAPE, how='left')
df_market_data = df_market_data[~df_market_data.index.duplicated(keep='first')]

# load CSM index data
data_path = 'data/index_data/'

with open(data_path + 'run4-mimicking.pkl', 'rb') as f:
    results = pickle.load(f)

trad_country_returns = results["trad_country_returns"]
trad_country_returns_EW = results["trad_country_returns_EW"]
csm_returns = results["csm_returns"]

trad_country_returns_excess = utils.calc_excess_returns(trad_country_returns, df_market_data, 'DTB4WK', log_returns=False)
trad_country_returns_EW_excess = utils.calc_excess_returns(trad_country_returns_EW, df_market_data, 'DTB4WK', log_returns=False)
csm_returns_excess = utils.calc_excess_returns(csm_returns, df_market_data, 'DTB4WK', log_returns=False)

trad_country_returns_excess_log = utils.calc_excess_returns(trad_country_returns, df_market_data, 'DTB4WK_LOG', log_returns=True)
trad_country_returns_EW_excess_log = utils.calc_excess_returns(trad_country_returns_EW, df_market_data, 'DTB4WK_LOG', log_returns=True)
csm_returns_excess_log = utils.calc_excess_returns(csm_returns, df_market_data, 'DTB4WK_LOG', log_returns=True)

# for VAR data, always use log returns!
data_VAR_full = df_market_data.join(
    csm_returns_excess_log.add_prefix('csm_'), how='inner'
).join(
    trad_country_returns_excess_log.add_prefix('trad_'), how='inner'
)

# Create local state variable dataframe
data_var_full_incl_local_state_vars = data_VAR_full.copy()
OECD_data = utils.load_OECD_data(
    path_and_filename='./data/OECD.SDD.STES,DSD_STES@DF_CLI,+all.csv'
)

CLI_LOG_SCALE = False
for country in OECD_data['Country'].unique():
    df = OECD_data[OECD_data['Country'] == country]
    df = df.rename(columns={'CLI': f'CLI_{country}'})

    if CLI_LOG_SCALE:
        df[f'CLI_{country}'] = np.log(df[f'CLI_{country}'])

    data_var_full_incl_local_state_vars = data_var_full_incl_local_state_vars.join(
        df.set_index('Date')[f'CLI_{country}'], how='left'
    )

# for robustness, use EW trad countries returns 
RC_data_VAR_full_EW = df_market_data.join(
    csm_returns_excess_log.add_prefix('csm_'), how='inner'
).join(
    trad_country_returns_EW_excess_log.add_prefix('trad_'), how='inner'
)

data_VAR_world = data_VAR_full.copy()
data_VAR_world['trad_wld'] = data_VAR_full[[col for col in data_VAR_full.columns if col.startswith('trad_')]].mean(axis=1)
data_VAR_world['csm_wld'] = data_VAR_full[[col for col in data_VAR_full.columns if col.startswith('csm_')]].mean(axis=1)
data_VAR_world = data_VAR_world[[
    'csm_wld',
    'trad_wld',
    'BAA10Y',
    'T10Y3M',
    'DTB3',
    'DTB4WK_LOG',
    'CAPE'
]]

COUNTRIES_OF_INTEREST = results['optim_description'].loc[results['optim_description']['Unnamed: 0'] == 'countries_for_optimization', '0'].values[0]
COUNTRIES_OF_INTEREST = ast.literal_eval(COUNTRIES_OF_INTEREST)



In [ ]:
dict_to_iterate = {
    'CSM index': csm_returns,
    'Trad country index': trad_country_returns
}
long_run_corr_results = {}

for index_type, return_series in dict_to_iterate.items():
    
    return_series_period1 = return_series.iloc[0:return_series.shape[0]//2, :]
    return_series_period2 = return_series.iloc[return_series.shape[0]//2:, :]

    cols = return_series.columns
    period1_pairs = []
    period2_pairs = []
    for a, b in combinations(cols, 2):
        period1_corr = return_series_period1[a].corr(return_series_period1[b])
        period2_corr = return_series_period2[a].corr(return_series_period2[b])

        period1_pairs.append({'pair': f'{a}-{b}', 'corr_period1': period1_corr})
        period2_pairs.append({'pair': f'{a}-{b}', 'corr_period2': period2_corr})

    pairwise_corrs_period1 = pd.DataFrame(period1_pairs).set_index('pair')
    pairwise_corrs_period2 = pd.DataFrame(period2_pairs).set_index('pair')

    # combined_corrs = pd.concat([pairwise_corrs_period1, pairwise_corrs_period2], axis=0)
    combined_corrs = pd.concat([pairwise_corrs_period1, pairwise_corrs_period2], axis=1)
    combined_corrs.sort_values(by='corr_period1', ascending=False)

    # Create a figure and axis
    fig, ax = plt.subplots()
    x = np.arange(len(combined_corrs))
    
    plt.scatter(x, combined_corrs['corr_period1'], alpha=0.7, s=30, color='tab:orange')
    plt.scatter(x, combined_corrs['corr_period2'], alpha=0.7, s=30, color='tab:blue')
    plt.axhline(y=(combined_corrs['corr_period1']).mean(), color='tab:orange', linestyle='--')
    plt.axhline(y=(combined_corrs['corr_period2']).mean(), color='tab:blue', linestyle='--')
    plt.xticks(x, combined_corrs.index, rotation=90)
    plt.legend(['Period 1', 'Period 2'], loc='best')
    plt.xlabel('Country pair')
    plt.ylabel('Correlation')
    plt.title(f'Pairwise correlations: Period 1 vs Period 2 (for {index_type})')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    model = stats.ttest_rel(
        pairwise_corrs_period1.to_numpy(), 
        pairwise_corrs_period2.to_numpy(), 
        alternative='greater'
    )

    fig.savefig(f'{output_path}fig_pairwise_corrs_{index_type.replace(" ", "_").lower()}.png', bbox_inches='tight')

    result = {
        'index_type': index_type,
        'mean_corr_period1': pairwise_corrs_period1.mean().values[0],
        'mean_corr_period2': pairwise_corrs_period2.mean().values[0],
        't_statistic': model.statistic[0],
        'p_value': model.pvalue[0],
        'df': model.df[0],
        #'matplotlib_figure': fig,
    }
    result_as_df = pd.DataFrame.from_dict(result, orient='index', columns=['value']).T

    if index_type == 'CSM index':
        long_run_corr_results['csm'] = result_as_df
    elif index_type == 'Trad country index':
        long_run_corr_results['trad'] = result_as_df



In [ ]:
spanning_results_separate_regressions = {}
for country in csm_returns_excess.columns:
    X = sm.add_constant(trad_country_returns_excess[country])
    y = csm_returns_excess[country]
    model = sm.OLS(y, X).fit()
    spanning_results_separate_regressions[country] = model

pd.set_option('display.float_format', lambda x: '%.6f' % x)

spanning_results_separate_regressions_results = {}
for test_asset, model in spanning_results_separate_regressions.items():
    # print(model.summary())
    result = {
        'coeffs': model.params,
        'std_errors': model.HC0_se,
        'tvalues': model.tvalues,
        'pvalues': model.pvalues,
        'r_squared': model.rsquared,
    }
    result = pd.DataFrame.from_dict(result).reset_index()
    spanning_results_separate_regressions_results[test_asset] = result

spanning_results_separate_regressions_results = pd.concat(
    {k: v.set_index('index') for k, v in spanning_results_separate_regressions_results.items()},
    names=['country', 'variable']
)

spanning_results_separate_regressions_results.reset_index()



In [ ]:
# input for multivariate GRS

spanning_results_multivariate_per_country = {}
for country in csm_returns_excess.columns:
    X = sm.add_constant(trad_country_returns_excess)
    y = csm_returns_excess[country]
    model = sm.OLS(y, X).fit()
    spanning_results_multivariate_per_country[country] = model



In [ ]:
## GRS test:

GRS_test_result = utils.grs_test(
    resid=np.array([model.resid for model in spanning_results_multivariate_per_country.values()]).T,
    alpha=np.array([model.params['const'] for model in spanning_results_multivariate_per_country.values()]).reshape(-1, 1),
    factors=np.array(trad_country_returns_excess)  
)

spanning_results_multivariate_per_country_GRS = pd.DataFrame({
    'GRS_F_statistic': GRS_test_result[0],
    'GRS_p_value': GRS_test_result[1]
}, index=[0])


In [ ]:
from utils import create_efficient_frontier

csm_mean_returns = csm_returns.mean().values
csm_cov_matrix = csm_returns.cov()

trad_mean_returns = trad_country_returns.mean().values
trad_cov_matrix = trad_country_returns.cov()

trad_mean_returns_EW = trad_country_returns_EW.mean().values
trad_cov_matrix_EW = trad_country_returns_EW.cov()

combined_returns = pd.concat([csm_returns, trad_country_returns], axis=1)
combined_mean_returns = combined_returns.mean().values
combined_cov_matrix = combined_returns.cov()

ef_csm = create_efficient_frontier(csm_mean_returns, csm_cov_matrix, n_points=150)
ef_trad = create_efficient_frontier(trad_mean_returns, trad_cov_matrix, n_points=150)
ef_trad_EW = create_efficient_frontier(trad_mean_returns_EW, trad_cov_matrix_EW, n_points=150)
ef_combined = create_efficient_frontier(combined_mean_returns, combined_cov_matrix, n_points=150)




In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.plot(ef_csm.volatility * 100, ef_csm.returns * 100, label='Efficient Frontier csm', color='blue', linestyle='-')
plt.plot(ef_trad.volatility * 100, ef_trad.returns * 100, label='Efficient Frontier trad', color='red', linestyle='-')
plt.plot(ef_trad_EW.volatility * 100, ef_trad_EW.returns * 100, label='Efficient Frontier trad (equal weighted)', color='orange', linestyle='--')
plt.plot(ef_combined.volatility * 100, ef_combined.returns * 100, label='Efficient Frontier combined', color='green', linestyle='--')
plt.xlabel('Portfolio Volatility (Std. Dev. of monthly returns %)')
plt.ylabel('Portfolio Return (monthly %)')
plt.legend(loc='upper left')
plt.grid(True)
plt.show()

fig.savefig(f'{output_path}fig_efficient_frontiers.png', bbox_inches='tight')

mean_variance_frontiers = { 
    'trad': ef_trad,
    'trad_EW': ef_trad_EW,
    'csm': ef_csm,
    'combined': ef_combined,
    #'matplotlib_figure': fig,
}


In [ ]:
import simulations

# load data
results_quantiles = pd.read_csv('data/simulations_results_quantiles_n_runs_10000.csv', index_col=0)
simulations.plot_sim_results(results_quantiles, metric="std_dev", n_runs=10000, list_of_quantiles=[0.5], facet_plot=False)

In [ ]:
### normalized by convergence values

import simulations
results_quantiles = pd.read_csv('data/simulations_results_quantiles_n_runs_10000.csv', index_col=0)

simulations.plot_sim_results(results_quantiles, metric="std_dev", n_runs=10000, list_of_quantiles=None, facet_plot=True, use_convergence_type=False, show_plot=False)
fig_simulations_std_dev = plt.gcf()
plt.close(fig_simulations_std_dev)

results_quantiles_normalized_convergence = simulations.get_convergence_normalized_results(results_quantiles=results_quantiles)

simulations.plot_sim_results(results_quantiles_normalized_convergence, metric="std_dev", n_runs=10000, list_of_quantiles=None, facet_plot=False, use_convergence_type=True, show_plot=False)
fig_simulations_std_dev_convergence = plt.gcf()
plt.close(fig_simulations_std_dev_convergence)


sim_results = {
    "fig_simulations_std_dev": fig_simulations_std_dev,
    "fig_simulations_std_dev_convergence": fig_simulations_std_dev_convergence,
}

fig_simulations_std_dev.savefig(f'{output_path}fig_simulations_std_dev.png', bbox_inches='tight')
fig_simulations_std_dev_convergence.savefig(f'{output_path}fig_simulations_std_dev_convergence.png', bbox_inches='tight')



In [ ]:
import simulations

# load data
results_quantiles = pd.read_csv('data/simulations_results_quantiles_n_runs_10000.csv', index_col=0)
simulations.plot_sim_results(results_quantiles, metric="std_dev", n_runs=10000, list_of_quantiles=None, facet_plot=True)

In [ ]:
import simulations

# load data
results_quantiles = pd.read_csv('data/simulations_results_quantiles_n_runs_10000.csv', index_col=0)
simulations.plot_sim_results(results_quantiles, metric="sharpe_ratio", n_runs=10000, list_of_quantiles=None, facet_plot=True)

We can visually interpret these results as follows. Let's first look at the change in standard deviation as the number of countries in portfolio increases. We are not directly interested in absolute levels (these are slightly higher on average for CSM indices), we want to look at the shape of each graph and compare that. So this graph, for the CSM indices, starts at around 6.8% (for 2 countries) and ends at 5.9%. This is a decrease of 14%. For the trad indices, the curve starts at 6% (for 2 countries) and ends at 5.4%. This is a decrease of 10%. Moreover, the curve for the traditional country indices seems to be slightly more concave, meaning it approaches the asymptote faster. 

These results suggests that the diversification benefits of traditional country indices are depleted faster. It takes fewer different countries in the index to obtain a decrease in portfolio risk similar to holding all countries. A flipside of this observation is that one could say that these traditional country indices are more alike. For the CSM indices, it takes more countries to approximate the portfolio standard deviation of the full set of countries. This suggests that CSM indices are more different from one another than traditional country indices are.

The statistics mentioned above in the text are summarized in the table below.

In [ ]:
results_quantiles = pd.read_csv('data/simulations_results_quantiles_n_runs_10000.csv', index_col=0)
csm_median_stddev_start = results_quantiles[(results_quantiles['dataset'] == 'csm') & (results_quantiles.index == 0.5) & (results_quantiles['num_countries'] == 2)]['std_dev']
csm_median_stddev_end = results_quantiles[(results_quantiles['dataset'] == 'csm') & (results_quantiles.index == 0.5) & (results_quantiles['num_countries'] == 16)]['std_dev']

trad_median_stddev_start = results_quantiles[(results_quantiles['dataset'] == 'trad') & (results_quantiles.index == 0.5) & (results_quantiles['num_countries'] == 2)]['std_dev']
trad_median_stddev_end = results_quantiles[(results_quantiles['dataset'] == 'trad') & (results_quantiles.index == 0.5) & (results_quantiles['num_countries'] == 16)]['std_dev']

pd.DataFrame({
    'CSM': 
    {
        'median_stddev_start (N=2)': csm_median_stddev_start.values[0],
        'median_stddev_end (N=16)': csm_median_stddev_end.values[0],
        'relative_change': csm_median_stddev_end.values[0] / csm_median_stddev_start.values[0] - 1
    },
    'Traditional': 
    {
        'median_stddev_start (N=2)': trad_median_stddev_start.values[0],
        'median_stddev_end (N=16)': trad_median_stddev_end.values[0],
        'relative_change': trad_median_stddev_end.values[0] / trad_median_stddev_start.values[0] - 1
    }
})


In [ ]:
data_VAR_world_trad = data_VAR_world[
    [
        "trad_wld",
        "BAA10Y",
        "T10Y3M",
        "DTB3",
        "CAPE"
    ]
]

N_CF, N_DR, results = var_decomposition.get_var_decomp(data_VAR_world_trad)

corr_mat = np.corrcoef(N_CF, N_DR)
std_N_CF = np.std(N_CF)
std_N_DR = np.std(N_DR)
np.fill_diagonal(corr_mat, [std_N_CF, std_N_DR])
campbell_DF_trad_world = pd.DataFrame(corr_mat, index=['CF', 'DR'], columns=['CF', 'DR']).round(4)
campbell_DF_trad_world

In [ ]:
data_VAR_world_csm = data_VAR_world[
    [
        "csm_wld",
        "BAA10Y",
        "T10Y3M",
        "DTB3",
        "CAPE"
    ]
]


N_CF, N_DR, results = var_decomposition.get_var_decomp(data_VAR_world_csm)

corr_mat = np.corrcoef(N_CF, N_DR)
std_N_CF = np.std(N_CF)
std_N_DR = np.std(N_DR)
np.fill_diagonal(corr_mat, [std_N_CF, std_N_DR])
campbell_DF_csm_world = pd.DataFrame(corr_mat, index=['CF', 'DR'], columns=['CF', 'DR']).round(4)
campbell_DF_csm_world






In [ ]:
import matplotlib.pyplot as plt

N_CF_trad_global, N_DR_trad_global, results_trad_global = var_decomposition.get_var_decomp(data_VAR_world_trad)
N_CF_csm_global, N_DR_csm_global, results_csm_global = var_decomposition.get_var_decomp(data_VAR_world_csm)

df_for_plot_world = pd.DataFrame({
    'Date': data_VAR_world_trad.reset_index()['Date'][:-1],
    'N_CF_trad_global': N_CF_trad_global,
    'N_DR_trad_global': N_DR_trad_global,
    'N_CF_csm_global': N_CF_csm_global,
    'N_DR_csm_global': N_DR_csm_global,
})

fig, axes = plt.subplots(2, 1, figsize=(16, 6), sharey=True)

facet_info = [
    ('N_CF_trad_global', 'N_DR_trad_global', 'Traditional Index'),
    ('N_CF_csm_global', 'N_DR_csm_global', 'CSM Index')
]

for ax, (cf_col, dr_col, title) in zip(axes, facet_info):
    ax.plot(df_for_plot_world['Date'], df_for_plot_world[cf_col], label='a. Cashflow News', color='tab:blue', linestyle='-')
    ax.plot(df_for_plot_world['Date'], df_for_plot_world[dr_col], label='b. Discount Rate News', color='tab:orange', linestyle='--')
    ax.set_title(title)
    # if ax is axes[0]:
    #     ax.set_xlabel('Date')
    ax.grid(True)
    ax.set_ylabel('News Component')
    ax.legend()

plt.suptitle('Cashflow and Discount Rate News: Trad and CSM')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

var_decomp_results_global = {
    'campbell_DF_trad_world': campbell_DF_trad_world,
    'campbell_DF_csm_world': campbell_DF_csm_world,
    'df_for_plot_world': df_for_plot_world
}


In [ ]:
result_dict_trad = {}
result_dict_csm = {}
for country in COUNTRIES_OF_INTEREST:
    data_VAR_trad = data_VAR_full[[
        f"trad_{country}",
        "BAA10Y",
        "T10Y3M",
        "DTB3",
        "CAPE"
    ]]

    data_VAR_csm = data_VAR_full[[
        f"csm_{country}",
        "BAA10Y",
        "T10Y3M",
        "DTB3",
        "CAPE"
    ]]

    trad_N_CF, trad_N_DR, trad_results = var_decomposition.get_var_decomp(data_VAR_trad)
    csm_N_CF, csm_N_DR, csm_results = var_decomposition.get_var_decomp(data_VAR_csm)

    trad_corr_mat = np.corrcoef(trad_N_CF, trad_N_DR)
    trad_std_N_CF = np.std(trad_N_CF)
    trad_std_N_DR = np.std(trad_N_DR)
    np.fill_diagonal(trad_corr_mat, [trad_std_N_CF, trad_std_N_DR])
    trad_corr_mat_df = pd.DataFrame(trad_corr_mat, index=['CF', 'DR'], columns=['CF', 'DR']).round(4)

    csm_corr_mat = np.corrcoef(csm_N_CF, csm_N_DR)
    csm_std_N_CF = np.std(csm_N_CF)
    csm_std_N_DR = np.std(csm_N_DR)
    np.fill_diagonal(csm_corr_mat, [csm_std_N_CF, csm_std_N_DR])
    csm_corr_mat_df = pd.DataFrame(csm_corr_mat, index=['CF', 'DR'], columns=['CF', 'DR']).round(4)

    result_dict_trad[country] = {
        'cash flow': trad_std_N_CF,
        'discount_rate': trad_std_N_DR
    }

    result_dict_csm[country] = {
        'cash flow': csm_std_N_CF,
        'discount_rate': csm_std_N_DR
    }

    # result_dict[country] = {
    #     'trad': {
    #         'corr_matrix': trad_corr_mat_df,
    #         # 'N_CF': trad_N_CF,
    #         # 'N_DR': trad_N_DR,
    #         # 'results': trad_results
    #     },
    #     'csm': {
    #         'corr_matrix': csm_corr_mat_df,
    #         # 'N_CF': csm_N_CF,
    #         # 'N_DR': csm_N_DR,
    #         # 'results': csm_results
    #     }
    # }

In [ ]:
df1 = pd.DataFrame.from_dict(result_dict_trad, orient='index', columns=['cash flow', 'discount_rate']).rename(columns={
    'cash flow': 'cash flow_trad',
    'discount_rate': 'discount_rate_trad'
})
df1['CF_to_DR_ratio_trad'] = df1['cash flow_trad'] - df1['discount_rate_trad']

df2 = pd.DataFrame.from_dict(result_dict_csm, orient='index', columns=['cash flow', 'discount_rate']).rename(columns={
    'cash flow': 'cash flow_csm',
    'discount_rate': 'discount_rate_csm'
})
df2['CF_to_DR_ratio_csm'] = df2['cash flow_csm'] - df2['discount_rate_csm']

df_combined = pd.concat([df1, df2], axis=1)
df_combined['CF_to_DR_ratio_diff'] = df_combined['CF_to_DR_ratio_csm'] - df_combined['CF_to_DR_ratio_trad']
df_combined

# df_combined['CF_to_DR_ratio_diff'].mean()

In [ ]:
# hypothesis testing

series_A = df_combined['CF_to_DR_ratio_trad'].to_numpy()
series_B = df_combined['CF_to_DR_ratio_csm'].to_numpy()

# t_statistic, p_value = stats.ttest_ind(series_A, series_B, equal_var=False)
t_statistic, p_value = stats.ttest_rel(series_B, series_A, alternative='two-sided')

print(f"T-statistic: {t_statistic}")
print(f"p-value: {p_value}")

t_test_CF_to_DR_ratio_diff = pd.DataFrame({
    't_statistic': t_statistic,
    'p_value': p_value
}, index=[0])


In [ ]:
# create descriptive statistics of VAR state variables of US trad
cols = [    
    'trad_US',
    'BAA10Y',
    "T10Y3M",
    "DTB3",
    "CAPE"
]

state_variables_descriptives_trad_US = data_VAR_full[cols].describe()

state_variables_descriptives_trad_US.loc['autocorr'] = data_VAR_full[cols].apply(lambda x: x.autocorr(), axis=0).T

state_variables_descriptives_trad_US.rename(columns={
    'trad_US': 'Return US_trad (log)',
    'BAA10Y': 'Default Spread',
    'T10Y3M': 'Term Spread',
    'DTB3': '3M Treasury Bill Rate',
    'CAPE': 'CAPE Ratio (log)'
}, inplace=True)

state_variables_descriptives_trad_US = state_variables_descriptives_trad_US.T.drop(columns=['count']).round(4)

# create coefficient table of VAR model for US trad
model = VAR(data_VAR_full[cols])

res = model.fit(maxlags=1, method='ols', trend='c') # explicitly parametrized, but these are the defaults used in the var_decomposition function

res_coefs = pd.DataFrame(
    res.coefs[0]
)
res_constants = pd.DataFrame(
    res.intercept
)

res_basic = pd.concat([res_constants, res_coefs], axis=1)

std_errors = pd.DataFrame(res.stderr).T
res_basic.columns = std_errors.columns

for col in res_basic.columns:
    res_basic[f'{col}_stderr'] = std_errors[col].values

var_coefficients_trad_US = res_basic.round(4)

# calculate R-squared for each equation in the VAR
var_r_squared_trad_US = {}
for col_name in cols:
    actual_values = data_VAR_full[col_name].iloc[res.k_ar:].values
    predicted_values = res.fittedvalues[col_name].values
    var_r_squared_trad_US[col_name] = r2_score(actual_values, predicted_values)

var_r_squared_trad_US = pd.DataFrame.from_dict(var_r_squared_trad_US, orient='index', columns=['R-squared'])


In [ ]:
#
#
# Other country: JP (robustness)
#
#
cols = [    
    'trad_JP',
    'BAA10Y',
    "T10Y3M",
    "DTB3",
    "CAPE"
]

state_variables_descriptives_trad_JP = data_VAR_full[cols].describe()

state_variables_descriptives_trad_JP.loc['autocorr'] = data_VAR_full[cols].apply(lambda x: x.autocorr(), axis=0).T

state_variables_descriptives_trad_JP.rename(columns={
    'trad_JP': 'Return JP_trad (log)',
    'BAA10Y': 'Default Spread',
    'T10Y3M': 'Term Spread',
    'DTB3': '3M Treasury Bill Rate',
    'CAPE': 'CAPE Ratio (log)'
}, inplace=True)

state_variables_descriptives_trad_JP = state_variables_descriptives_trad_JP.T.drop(columns=['count']).round(4)

# create coefficient table of VAR model for JP trad
model = VAR(data_VAR_full[cols])

res = model.fit(maxlags=1, method='ols', trend='c') # explicitly parametrized, but these are the defaults used in the var_decomposition function

res_coefs = pd.DataFrame(
    res.coefs[0]
)
res_constants = pd.DataFrame(
    res.intercept
)

res_basic = pd.concat([res_constants, res_coefs], axis=1)

std_errors = pd.DataFrame(res.stderr).T
res_basic.columns = std_errors.columns

for col in res_basic.columns:
    res_basic[f'{col}_stderr'] = std_errors[col].values

var_coefficients_trad_JP = res_basic.round(4)

# calculate R-squared for each equation in the VAR
var_r_squared_trad_JP = {}
for col_name in cols:
    actual_values = data_VAR_full[col_name].iloc[res.k_ar:].values
    predicted_values = res.fittedvalues[col_name].values
    var_r_squared_trad_JP[col_name] = r2_score(actual_values, predicted_values)

var_r_squared_trad_JP = pd.DataFrame.from_dict(var_r_squared_trad_JP, orient='index', columns=['R-squared'])

var_r_squared_trad_JP

In [ ]:
# including local state variables

countries_with_lcl_data = [col[-2:] for col in data_var_full_incl_local_state_vars.columns if col.startswith('CLI_')]
result_with_lcl_dict_trad = {}
result_with_lcl_dict_csm = {}
result_differences = {}
for country in countries_with_lcl_data:
    df_trad = data_var_full_incl_local_state_vars[[
        f"trad_{country}",
        "BAA10Y",
        "T10Y3M",
        "DTB3",
        "CAPE",
    ]]

    df_with_lcl_trad = data_var_full_incl_local_state_vars[[
        f"trad_{country}",
        "BAA10Y",
        "T10Y3M",
        "DTB3",
        "CAPE",
        f"CLI_{country}"
    ]]

    df_csm = data_var_full_incl_local_state_vars[[
        f"csm_{country}",
        "BAA10Y",
        "T10Y3M",
        "DTB3",
        "CAPE",
    ]]

    df_with_lcl_csm = data_var_full_incl_local_state_vars[[
        f"csm_{country}",
        "BAA10Y",
        "T10Y3M",
        "DTB3",
        "CAPE",
        f"CLI_{country}"
    ]]

    trad_N_CF, trad_N_DR, trad_results = var_decomposition.get_var_decomp(df_trad)
    trad_N_lcl_CF, trad_N_lcl_DR, trad_lcl_results = var_decomposition.get_var_decomp(df_with_lcl_trad)
    csm_N_CF, csm_N_DR, csm_results = var_decomposition.get_var_decomp(df_csm)
    csm_N_lcl_CF, csm_N_lcl_DR, csm_lcl_results = var_decomposition.get_var_decomp(df_with_lcl_csm)

    trad_corr_mat = np.corrcoef(trad_N_CF, trad_N_DR)
    trad_std_N_CF = np.std(trad_N_CF)
    trad_std_N_DR = np.std(trad_N_DR)
    np.fill_diagonal(trad_corr_mat, [trad_std_N_CF, trad_std_N_DR])
    trad_corr_mat_df = pd.DataFrame(trad_corr_mat, index=['CF', 'DR'], columns=['CF', 'DR']).round(4)

    trad_lcl_corr_mat = np.corrcoef(trad_N_lcl_CF, trad_N_lcl_DR)
    trad_lcl_std_N_CF = np.std(trad_N_lcl_CF)
    trad_lcl_std_N_DR = np.std(trad_N_lcl_DR)
    np.fill_diagonal(trad_lcl_corr_mat, [trad_lcl_std_N_CF, trad_lcl_std_N_DR])
    trad_lcl_corr_mat_df = pd.DataFrame(trad_lcl_corr_mat, index=['CF', 'DR'], columns=['CF', 'DR']).round(4)

    csm_corr_mat = np.corrcoef(csm_N_CF, csm_N_DR)
    csm_std_N_CF = np.std(csm_N_CF)
    csm_std_N_DR = np.std(csm_N_DR)
    np.fill_diagonal(csm_corr_mat, [csm_std_N_CF, csm_std_N_DR])
    csm_corr_mat_df = pd.DataFrame(csm_corr_mat, index=['CF', 'DR'], columns=['CF', 'DR']).round(4)

    csm_lcl_corr_mat = np.corrcoef(csm_N_lcl_CF, csm_N_lcl_DR)
    csm_lcl_std_N_CF = np.std(csm_N_lcl_CF)
    csm_lcl_std_N_DR = np.std(csm_N_lcl_DR)
    np.fill_diagonal(csm_lcl_corr_mat, [csm_lcl_std_N_CF, csm_lcl_std_N_DR])
    csm_lcl_corr_mat_df = pd.DataFrame(csm_lcl_corr_mat, index=['CF', 'DR'], columns=['CF', 'DR']).round(4)


    # differences between news vectors
    trad_std_N_CF_diff = np.std(np.array(trad_N_CF) - np.array(trad_N_lcl_CF))
    trad_std_N_DR_diff = np.std(np.array(trad_N_lcl_DR) - np.array(trad_N_DR))
    csm_std_N_CF_diff = np.std(np.array(csm_N_lcl_CF) - np.array(csm_N_CF))
    csm_std_N_DR_diff = np.std(np.array(csm_N_lcl_DR) - np.array(csm_N_DR))    

    result_with_lcl_dict_trad[country] = {
        'cash flow': trad_lcl_std_N_CF,
        'discount_rate': trad_lcl_std_N_DR
    }

    result_with_lcl_dict_csm[country] = {
        'cash flow': csm_lcl_std_N_CF,
        'discount_rate': csm_lcl_std_N_DR
    }

    result_differences[country] = {
        'trad_std_N_CF_diff': trad_std_N_CF_diff,
        'trad_std_N_DR_diff': trad_std_N_DR_diff,
        'csm_std_N_CF_diff': csm_std_N_CF_diff,
        'csm_std_N_DR_diff': csm_std_N_DR_diff
    }

    # result_dict[country] = {
    #     'trad': {
    #         'corr_matrix': trad_corr_mat_df,
    #         # 'N_CF': trad_N_CF,
    #         # 'N_DR': trad_N_DR,
    #         # 'results': trad_results
    #     },
    #     'csm': {
    #         'corr_matrix': csm_corr_mat_df,
    #         # 'N_CF': csm_N_CF,
    #         # 'N_DR': csm_N_DR,
    #         # 'results': csm_results
    #     }
    # }



In [ ]:
# for country in result_with_lcl_dict_trad.keys():
#     print(f"Country: {country}")
#     print("Traditional Index:")
#     print(pd.DataFrame([
#             result_with_lcl_dict_trad[country],
#             result_with_lcl_dict_csm[country]
#     ], index=['Traditional', 'CSM']))


for country in result_differences.keys():
    print(f"Country: {country}")
    print(pd.DataFrame([
            result_differences[country]
    ], index=[" "]).T)
    print( ) 



In [ ]:
data_to_export = {
    'long_run_corr_results': long_run_corr_results,
    'mean_variance_frontiers': mean_variance_frontiers,
    'spanning_results_separate_regressions_results': spanning_results_separate_regressions_results,
    'spanning_results_multivariate_per_country_GRS': spanning_results_multivariate_per_country_GRS,
    'state_variables_descriptives_trad_US': state_variables_descriptives_trad_US,
    'var_coefficients_trad_US': var_coefficients_trad_US,
    'var_r_squared_trad_US': var_r_squared_trad_US,
    'state_variables_descriptives_trad_JP': state_variables_descriptives_trad_JP,
    'var_coefficients_trad_JP': var_coefficients_trad_JP,
    'var_r_squared_trad_JP': var_r_squared_trad_JP,
    'var_decomp_summary': df_combined,
    't_test_CF_to_DR_ratio_diff': t_test_CF_to_DR_ratio_diff,    
    'var_decomp_results_global': var_decomp_results_global,
}

with open('rmarkdown/run4-mimicking_addl.pkl', 'wb') as f:
    pickle.dump(data_to_export, f)

In [ ]:
### convert output to csv's for easier use in R markdown


# write dataframes in nested dicts to csv
for k, v in data_to_export.items():
    if isinstance(v, pd.DataFrame):
        v.to_csv(f'{output_path}/{k}.csv')
    elif isinstance(v, dict):
        for k2, v2 in v.items():
            if isinstance(v2, pd.DataFrame):
                v2.to_csv(f'{output_path}/{k}_{k2}.csv')            
            elif isinstance(v2, dict):
                for k3, v3 in v2.items():
                    if isinstance(v3, pd.DataFrame):
                        v3.to_csv(f'{output_path}/{k}_{k2}_{k3}.csv')
                    else:
                        print(f'warning! incomplete export for {k}, {k2}, {k3}')
            else:
                print(f'warning! incomplete export for {k}, {k2}')
    else:
        print(f'warning! incomplete export for {k}')



    







In [ ]:
COUNTRIES_OF_INTEREST
